In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash, waterdewpoint
from neqsim.process import stream, separator, compressor, heater, cooler
import neqsim.jneqsim as jneqsim
import matplotlib.pyplot as plt
import pandas as pd

## 1. Introduction to TEG Dehydration

Natural gas from the reservoir contains water vapor. This water must be removed to:
- Prevent hydrate formation
- Prevent corrosion
- Meet pipeline specifications

TEG (Triethylene Glycol) is the most common dehydration agent because:
- High affinity for water
- Can be regenerated and recycled
- Low vapor pressure (low losses)
- Thermally stable

## 2. Create Wet Natural Gas

In [ ]:
# Create wet natural gas using CPA equation of state (handles water well)
wet_gas = fluid("cpa")
wet_gas.addComponent("nitrogen", 1.0, "mol%")
wet_gas.addComponent("CO2", 2.0, "mol%")
wet_gas.addComponent("methane", 85.0, "mol%")
wet_gas.addComponent("ethane", 6.0, "mol%")
wet_gas.addComponent("propane", 3.0, "mol%")
wet_gas.addComponent("n-butane", 1.0, "mol%")
wet_gas.addComponent("water", 2.0, "mol%")  # Saturated with water
wet_gas.setMixingRule(10)  # CPA mixing rule
wet_gas.setMultiPhaseCheck(True)

print("Wet gas composition created")

## 3. Calculate Water Dew Point Before Dehydration

In [ ]:
# Set process conditions
wet_gas.setTemperature(30.0, "C")
wet_gas.setPressure(70.0, "bara")
TPflash(wet_gas)

# Calculate water dew point
wdp_before = waterdewpoint(wet_gas)

print(f"Process conditions: T = 30°C, P = 70 bara")
print(f"Water dew point of inlet gas: {wdp_before:.1f}°C")
print(f"\nNote: Gas will form liquid water if cooled below {wdp_before:.1f}°C")

## 4. Create TEG Lean Glycol Stream

In [ ]:
# Create lean TEG stream (regenerated glycol)
# Typical lean TEG purity is 99.0-99.9 wt%
lean_teg = fluid("cpa")
lean_teg.addComponent("TEG", 0.99, "kg/hr")  # 99% TEG
lean_teg.addComponent("water", 0.01, "kg/hr")  # 1% water
lean_teg.setMixingRule(10)
lean_teg.setTemperature(40.0, "C")  # TEG inlet temperature
lean_teg.setPressure(70.0, "bara")

print("Lean TEG stream:")
print(f"  TEG purity: 99 wt%")
print(f"  Temperature: 40°C")
print(f"  Pressure: 70 bara")

## 5. Water Content vs TEG Concentration

The water content in dried gas depends on TEG purity and circulation rate.

In [ ]:
# Simulate effect of TEG purity on water dew point depression
teg_purities = [98.0, 99.0, 99.5, 99.9]
dew_point_depression = []

print("Effect of TEG purity on water removal:")
print("\nTEG Purity [wt%] | Achievable Dew Point Depression [°C]")
print("-" * 50)

# Empirical correlation for dew point depression
# Higher purity = lower equilibrium water content
for purity in teg_purities:
    # Approximate dew point depression (simplified)
    if purity >= 99.9:
        dp_dep = 60
    elif purity >= 99.5:
        dp_dep = 50
    elif purity >= 99.0:
        dp_dep = 40
    else:
        dp_dep = 30
    
    dew_point_depression.append(dp_dep)
    print(f"{purity:16.1f} | {dp_dep}")

In [ ]:
# Plot effect of TEG purity
plt.figure(figsize=(8, 5))
plt.plot(teg_purities, dew_point_depression, 'bo-', markersize=10, linewidth=2)
plt.xlabel('TEG Purity [wt%]', fontsize=12)
plt.ylabel('Dew Point Depression [°C]', fontsize=12)
plt.title('Effect of TEG Purity on Dehydration Performance', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim(0, 70)
plt.tight_layout()
plt.show()

## 6. TEG Process Description

A typical TEG dehydration process consists of:

1. **Inlet Separator**: Remove free liquids from wet gas
2. **Contactor (Absorber)**: Counter-current contact of gas and TEG
3. **Flash Drum**: Recover absorbed hydrocarbons
4. **Regenerator (Reboiler)**: Boil off water from rich TEG
5. **Surge Drum**: Buffer for lean TEG
6. **Pump**: Circulate TEG back to contactor

In [ ]:
# Process schematic (text-based)
print("""
TEG DEHYDRATION PROCESS FLOW DIAGRAM
======================================

                    Dry Gas Out
                         ↑
                    ┌─────────┐
   Lean TEG ──────►│         │
                    │ CONTACTOR│
                    │ (Absorber)│
   Wet Gas ──────►│         │
                    └────┬────┘
                         │ Rich TEG
                         ▼
                    ┌─────────┐        Water Vapor
                    │  FLASH  │           ↑
                    │  DRUM   │      ┌─────────┐
                    └────┬────┘      │         │
                         │           │ REGEN.  │
                         └──────────►│ (Still) │
                                     │         │
                    ┌─────────┐      └────┬────┘
   ◄────────────────│  PUMP   │◄─────────┘
   Lean TEG         └─────────┘    Lean TEG
""")

## 7. TEG Circulation Rate

The TEG circulation rate is a key design parameter:
- Too low: Insufficient drying
- Too high: Higher energy consumption, larger equipment

In [ ]:
# Typical TEG circulation rates
print("Typical TEG Circulation Rates:")
print("\nApplication              | TEG Rate [L TEG/kg H2O removed]")
print("-" * 60)
print("Standard dehydration     | 15-25")
print("Deep dehydration         | 25-40")
print("LNG feed gas             | 40-60")

# Calculate for our example
gas_flow_mmscfd = 100  # Million standard cubic feet per day
water_content_inlet = 1000  # mg/Sm³
water_content_outlet = 50  # mg/Sm³ (target)

# Water removed per day
water_removed_kg_day = (water_content_inlet - water_content_outlet) * gas_flow_mmscfd * 28316.8 / 1e6
print(f"\nExample calculation for {gas_flow_mmscfd} MMscfd gas:")
print(f"  Water removed: {water_removed_kg_day:.1f} kg/day")

## 8. Regeneration Temperature

The reboiler temperature affects TEG purity and degradation.

In [ ]:
# TEG thermal limits
print("TEG Regeneration Guidelines:")
print("\nReboiler Temperature | Effect")
print("-" * 50)
print("< 175°C              | Low TEG purity (< 98.5%)")
print("175-190°C            | Standard operation (99.0-99.5%)")
print("190-204°C            | High purity (99.5-99.9%)")
print("> 204°C              | TEG degradation begins!")

print("\n⚠️ Warning: TEG decomposes above 204°C (400°F)")
print("   Use stripping gas for very high purity requirements")

## 9. Water Dew Point Specifications

In [ ]:
# Typical water dew point specifications
specs = {
    "Pipeline (warm climate)": -10,
    "Pipeline (cold climate)": -20,
    "Offshore export": -18,
    "LNG feed gas": -60,
    "CNG": -40,
}

print("Water Dew Point Specifications:")
print("\nApplication              | Max Water Dew Point [°C]")
print("-" * 50)
for app, wdp in specs.items():
    print(f"{app:24} | {wdp}")

In [ ]:
# Visualize specifications
plt.figure(figsize=(10, 5))
apps = list(specs.keys())
wdps = list(specs.values())

colors = ['green' if w >= -20 else 'orange' if w >= -40 else 'red' for w in wdps]
plt.barh(apps, wdps, color=colors)
plt.xlabel('Water Dew Point [°C]', fontsize=12)
plt.title('Water Dew Point Specifications by Application', fontsize=14)
plt.axvline(x=0, color='black', linestyle='--', alpha=0.5)
plt.xlim(-70, 5)
plt.tight_layout()
plt.show()

## Summary

This notebook covered:
- Why gas dehydration is necessary
- How TEG dehydration works
- Key design parameters:
  - TEG purity (99.0-99.9 wt%)
  - Circulation rate (15-60 L/kg water)
  - Reboiler temperature (175-204°C)
- Water dew point specifications for different applications

**For more detailed process simulation, see the NeqSim process module examples.**